In [1]:
import os,re,glob
import pandas as pd
import numpy as np
from collections import Counter


users_dir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47"
prev_users_dir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction"
specie = "human"
GENOMEDIR = "/data/genomics/marta/genomes"


Bams are processed by Sara --> look for the scripts

Outputs are in:

`/users/genomics/saraa/projectTestis/STAR/{human/macaca/mouse}`

## Bam2Fastq + QC + STAR shiva

/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/macaca/STAR for macaca

/datasets/marta/Wang2020/RNASeq/mouse* for mouse


In [20]:
#!/bin/bash

#SBATCH -p long,bigmem            # Partition to submit to
#SBATCH --cpus-per-task=8
#SBATCH --mem-per-cpu 10Gb           # Memory in MB
#SBATCH -J stringtie           # job name
#SBATCH -o /users/genomics/marta/logs/stringtie_%A_%a.out    # File to which standard out will be written
#SBATCH -e /users/genomics/marta/logs/stringtie_%A_%a.err    # File to which standard err will be written

####Change output and output error paths to redirect the log files generated

#export PATH=/genomics/users/irubia/tools/stringtie2/:$PATH
specie="human"
declare -a bam_array


BAMDIR=/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/$specie/STAR
for file in $BAMDIR/*bam; do
    bam_array=(${bam_array[@]} $file) # Add new element at the end of the array
done



i=$(($SLURM_ARRAY_TASK_ID -1))
specie="human"
OUTDIR=/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/$specie/stringtieReconstruction
mkdir -p $OUTDIR
bam=${bam_array[i]}
echo $bam

if [ "$specie" == "macaca" ]; then
  REF=/datasets/Common/ReferenceGenomes/Macaca/10.111/GenomesAndAnnotation/Macaca_mulatta.Mmul_10.111.gtf
elif [ "$specie" == "mouse" ]; then
  REF=/datasets/Common/ReferenceGenomes/Mouse/mm39/GenomesAndAnnotation/gencode.vM32.annotation.gtf
elif [ "$specie" == "human" ]; then
  # REF=/datasets/Common/ReferenceGenomes/Human/GRCh38/GenomesAndAnnotation/gencode.v38.primary_assembly.annotation.gtf
  REF=/data/genomics/marta/genomes/Annot_files_GTF/GENCODE_47_M36/gencode.v47.primary_assembly.annotation.fixed.gtf
elif [ "$specie" == "platypus" ]; then
  REF=/datasets/Common/ReferenceGenomes/Platypus/mOrnaAna1/GenomesAndAnnotation/Ornithorhynchus_anatinus.mOrnAna1.p.v1.112.chr.gtf
elif [ "$specie" == "opossum" ]; then
  REF=/datasets/Common/ReferenceGenomes/Opossum/ASSM229v1/GenomesAndAnnotation/Monodelphis_domestica.ASM229v1.112.chr.gtf
elif [ "$specie" == "chicken" ]; then
  REF=/datasets/Common/ReferenceGenomes/Chicken/GRCg7b/GenomesAndAnnotation/Gallus_gallus.bGalGal1.mat.broiler.GRCg7b.112.chr.gtf
fi

sample=${bam%%Aligned*}
sample=${sample##*/}

module load stringtie/2.2.0
## in this case, we assume it's always fr
stringtie $bam -G $REF --conservative --fr -o $OUTDIR/${sample}.gtf -C $OUTDIR/${sample}_cov.gtf


SyntaxError: invalid syntax (2449622322.py, line 14)

#### Make summary

In [2]:
%%bash -s "$users_dir" "$specie"
## Total transcriptome + Presumably annotated + Presumably novel
OUT=$1/$2/stringtieReconstruction/transcriptomeReconstruction.csv
echo $OUT
if [ -f "$OUT" ] ; then
    rm "$OUT"
fi
echo -e "Sample\tTotal_transcripts\tAnnotated_transcripts\tNovel_transcript" >> $OUT

for file in $1/$2/stringtieReconstruction/*gtf; do
    if [[ ${file} != *"cov"* ]]; then 
        name="${file%%.*}"
        name="${name##*/}"
        echo -e $name"\t"$(awk '($3 == "transcript") {print}' $file | wc -l)"\t"$(awk '($3 == "transcript") {print}' $file | grep "reference" | wc -l)"\t"$(awk '($3 == "transcript") {print}' $file | grep -v "reference" | wc -l) >> $OUT
    fi
done


/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/stringtieReconstruction/transcriptomeReconstruction.csv


From here, I **need** to know if what is considered as novel is really novel. Intersect with annotated genes. Those that do not overlap at all, are novels for sure: maintain.

For **human**:

`$GENOMEDIR/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf`


`$GENOMEDIR/Annot_files_GTF/GENCODE_47_M36/gencode.v47.primary_assembly.annotation.fixed.gtf`

For **macaca**:

`/datasets/Common/ReferenceGenomes/Macaca/10.111/GenomesAndAnnotation/Macaca_mulatta.Mmul_10.111.bed`

In [3]:
%%bash -s "$users_dir" "$specie"

OUTDIR=$1/$2/stringtieReconstruction

mkdir $OUTDIR/NOoriented
mkdir $OUTDIR/oriented

for file in $OUTDIR/*.gtf; do 
    if [[ ${file} != *"cov"* ]]; then 
        if [[ ${file} != *"gene"* ]]; then 
            name=${file##*/}
            name=${name%%.*}
            #separation of novel transcripts oriented not oriented
            awk '($7 == ".") {print}' $file > $OUTDIR/NOoriented/${name}_NOoriented.gtf
            awk '($7 != ".") {print}' $file > $OUTDIR/oriented/${name}_oriented.gtf
        fi
    fi
done


**FORCING STRANDNESS**

For the novel transcripts oriented, we force orientation when checking the overlapping with the reference.

In [ ]:
%%bash -s "$users_dir" "$specie" "$GENOMEDIR"

OUTDIR=$1/$2/stringtieReconstruction

module load BEDTools/2.30.0-GCC-10.2.0

if [ "$2" == "macaca" ]; then
  REF=/datasets/Common/ReferenceGenomes/Macaca/10.111/GenomesAndAnnotation/Macaca_mulatta.Mmul_10.111.gtf
elif [ "$2" == "mouse" ]; then
  REF=/datasets/Common/ReferenceGenomes/Mouse/mm39/GenomesAndAnnotation/gencode.vM32.annotation.gtf
elif [ "$2" == "human" ]; then
  REF=/data/genomics/marta/genomes/Annot_files_GTF/GENCODE_47_M36/gencode.v47.primary_assembly.annotation.fixed.gtf
elif [ "$2" == "platypus" ]; then
  REF=/datasets/Common/ReferenceGenomes/Platypus/mOrnaAna1/GenomesAndAnnotation/Ornithorhynchus_anatinus.mOrnAna1.p.v1.112.chr.gtf
elif [ "$2" == "opossum" ]; then
  REF=/datasets/Common/ReferenceGenomes/Opossum/ASSM229v1/GenomesAndAnnotation/Monodelphis_domestica.ASM229v1.112.chr.gtf
elif [ "$2" == "chicken" ]; then
  REF=/datasets/Common/ReferenceGenomes/Chicken/GRCg7b/GenomesAndAnnotation/Gallus_gallus.bGalGal1.mat.broiler.GRCg7b.112.chr.gtf
fi

for file in $OUTDIR/oriented/*oriented.gtf; do 
    name=${file##*/}
    name=${name%%.*}
    #select novel transcripts that do not overlap at all with the gencode annotation (-v)
    bedtools intersect -nonamecheck -v -s -a $file -b $REF > $OUTDIR/oriented/${name}_novel.gtf
done


***** WARNING: File /data/genomics/marta/genomes/Annot_files_GTF/GENCODE_47_M36/gencode.v47.primary_assembly.annotation.fixed.gtf has inconsistent naming convention for record:
GL000195.1	HAVANA	gene	2579	87094	.	-	.	gene_id "ENSG00000303853.1"; ID "ENSG00000303853.1"; gene_name "ENSG00000303853"; gene_type "lncRNA"; level "2"; tag "ncRNA_host" "overlapping_locus";

***** WARNING: File /data/genomics/marta/genomes/Annot_files_GTF/GENCODE_47_M36/gencode.v47.primary_assembly.annotation.fixed.gtf has inconsistent naming convention for record:
GL000195.1	HAVANA	gene	2579	87094	.	-	.	gene_id "ENSG00000303853.1"; ID "ENSG00000303853.1"; gene_name "ENSG00000303853"; gene_type "lncRNA"; level "2"; tag "ncRNA_host" "overlapping_locus";

***** WARNING: File /data/genomics/marta/genomes/Annot_files_GTF/GENCODE_47_M36/gencode.v47.primary_assembly.annotation.fixed.gtf has inconsistent naming convention for record:
GL000195.1	HAVANA	gene	2579	87094	.	-	.	gene_id "ENSG00000303853.1"; ID "ENSG00000303

**WITHOUT FORCING STRANDNESS**

For the novel transcripts NOT oriented, we do not force orientation when checking the overlapping with the reference.

In [5]:
%%bash -s "$users_dir" "$specie" "$GENOMEDIR"

OUTDIR=$1/$2/stringtieReconstruction

module load BEDTools/2.30.0-GCC-10.2.0

if [ "$2" == "macaca" ]; then
  REF=/datasets/Common/ReferenceGenomes/Macaca/10.111/GenomesAndAnnotation/Macaca_mulatta.Mmul_10.111.gtf
elif [ "$2" == "mouse" ]; then
  REF=/datasets/Common/ReferenceGenomes/Mouse/mm39/GenomesAndAnnotation/gencode.vM32.annotation.gtf
elif [ "$2" == "human" ]; then
  REF=/data/genomics/marta/genomes/Annot_files_GTF/GENCODE_47_M36/gencode.v47.primary_assembly.annotation.fixed.gtf
elif [ "$2" == "platypus" ]; then
  REF=/datasets/Common/ReferenceGenomes/Platypus/mOrnaAna1/GenomesAndAnnotation/Ornithorhynchus_anatinus.mOrnAna1.p.v1.112.chr.gtf
elif [ "$2" == "opossum" ]; then
  REF=/datasets/Common/ReferenceGenomes/Opossum/ASSM229v1/GenomesAndAnnotation/Monodelphis_domestica.ASM229v1.112.chr.gtf
elif [ "$2" == "chicken" ]; then
  REF=/datasets/Common/ReferenceGenomes/Chicken/GRCg7b/GenomesAndAnnotation/Gallus_gallus.bGalGal1.mat.broiler.GRCg7b.112.chr.gtf
fi

for file in $OUTDIR/NOoriented/*oriented.gtf; do 
    name=${file##*/}
    name=${name%%.*}

    #select novel transcripts that do not overlap at all with the gencode annotation (-v)
    bedtools intersect -nonamecheck -v -a $file -b $REF > $OUTDIR/NOoriented/${name}_novel.gtf
done

I want to see if there are "alone exons", I don't want them, only full transcripts. Check transcript line, store geneid (STRG.XX), see if later exons contain the same. If so, store in new file, if not, skip.

In [6]:
for file in os.listdir(os.path.join(users_dir,specie,"stringtieReconstruction/NOoriented")):
    if file.endswith("novel.gtf"):
        outname=file[:-4]+"_complete.gtf"
        with open(os.path.join(os.path.join(users_dir,specie,"stringtieReconstruction/NOoriented"),file)) as inp:
            with open(os.path.join(os.path.join(users_dir,specie,"stringtieReconstruction/NOoriented"),outname), 'w') as out:
                lines = inp.readlines()
                t_gene_id = ""
                for line in lines:
                    tab_split_line=line.split("\t")
                    coma_split_line=line.split(";")
                    gene_id=re.findall(r'"([^"]*)"',coma_split_line[0])
                    if tab_split_line[2] == "transcript":
                        t_gene_id = gene_id[0]
                        out.write(line)
                    if tab_split_line[2] == "exon":
                        if gene_id[0] == t_gene_id:
                            out.write(line)
                        else:
                            continue


In [7]:
for file in os.listdir(os.path.join(users_dir,specie,"stringtieReconstruction/oriented")):
    if file.endswith("novel.gtf"):
        outname=file[:-4]+"_complete.gtf"
        with open(os.path.join(os.path.join(users_dir,specie,"stringtieReconstruction/oriented"),file)) as inp:
            with open(os.path.join(os.path.join(users_dir,specie,"stringtieReconstruction/oriented"),outname), 'w') as out:
                lines = inp.readlines()
                t_gene_id = ""
                for line in lines:
                    tab_split_line=line.split("\t")
                    coma_split_line=line.split(";")
                    gene_id=re.findall(r'"([^"]*)"',coma_split_line[0])
                    if tab_split_line[2] == "transcript":
                        t_gene_id = gene_id[0]
                        out.write(line)
                    if tab_split_line[2] == "exon":
                        if gene_id[0] == t_gene_id:
                            out.write(line)
                        else:
                            continue


In [8]:
%%bash -s "$users_dir" "$specie" "$GENOMEDIR"

## Concatenate both oriented and unoriented
OUTDIR=$1/$2/stringtieReconstruction/intersect_novel
mkdir $OUTDIR


for sample in $(cat $1/$2/stringtieReconstruction/transcriptomeReconstruction.csv | tail -n +2 | cut -f1); do
    #concatenate both results in order to obtain a gtf file with all novel transcripts considered
    cat $1/$2/stringtieReconstruction/oriented/${sample}_oriented_novel_complete.gtf $1/$2/stringtieReconstruction/NOoriented/${sample}_NOoriented_novel_complete.gtf > $OUTDIR/${sample}_novel.gtf
done


mkdir: cannot create directory ‘/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/stringtieReconstruction/intersect_novel’: File exists


In [9]:
%%bash -s "$users_dir" "$specie"
## Total transcriptome + Presumably annotated + Presumably novel
OUT=$1/$2/stringtieReconstruction/clean_transcriptomeReconstruction.csv
OUTDIR=$1/$2/stringtieReconstruction/intersect_novel

echo $OUT
if [ -f "$OUT" ] ; then
    rm "$OUT"
fi
echo -e "Sample\tNovel_transcript" >> $OUT

for file in $OUTDIR/*_novel.gtf; do
        name="${file%%_novel.*}"
        name="${name##*/}"
        echo -e $name"\t"$(awk '($3 == "transcript") {print}' $file | wc -l) >> $OUT
done


/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/stringtieReconstruction/clean_transcriptomeReconstruction.csv


Now we use gffcompare so that we unify all novel transcripts into a single transcriptome

In [10]:
%%bash -s "$users_dir" "$specie"

OUTDIR=$1/$2/gffcompare
mkdir $OUTDIR

module load gffcompare/0.12.2

ls $1/$2/stringtieReconstruction/intersect_novel/*_novel.gtf > $OUTDIR/list_novel_assembled_files_stranded.txt

cd $OUTDIR
gffcompare -i list_novel_assembled_files_stranded.txt -o gffcmp.assembled.stranded


    

mkdir: cannot create directory ‘/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/gffcompare’: File exists
Loading query file #1: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/stringtieReconstruction/intersect_novel/human_brain_rna_1_novel.gtf
Loading query file #2: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/stringtieReconstruction/intersect_novel/human_brain_rna_2_novel.gtf
Loading query file #3: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/stringtieReconstruction/intersect_novel/human_brain_rna_3_novel.gtf
Loading query file #4: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/stringtieReconstruction/intersect_novel/human_liver_rna_1_novel.gtf
Loading query file #5: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/stringtieR

In [11]:
%%bash -s "$users_dir" "$specie"

echo -e "\n"$proj
awk '$3 == "transcript" {print}' $1/$2/gffcompare/gffcmp.assembled.stranded.combined.gtf | cut -f 7 | sort | uniq -c


    570 +
    613 -


Let's create a new reference file with novel reconstructed transcripts

In [12]:
%%bash -s "$users_dir" "$specie" "$GENOMEDIR"

export PATH=/data/genomics/marta/tools/gff3sort-master:$PATH

## reference 
REF=$3/Annot_files_GTF/GENCODE_47_M36/gencode.v47.primary_assembly.annotation.fixed.gtf 

OUTDIR=$1/$2/newReference_Resconstructed
mkdir $OUTDIR

## gffcompare
gffcompare=$1/$2/gffcompare/gffcmp.assembled.stranded.combined.gtf

## concat
cat $gffcompare $REF > $OUTDIR/gencode.v47.gffcompare.TestisLiverBrain.annotation.gtf

#sort the newly assembled transcriptome
gff3sort.pl --precise --chr_order natural $OUTDIR/gencode.v47.gffcompare.TestisLiverBrain.annotation.gtf > $OUTDIR/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.gtf
rm $OUTDIR/gencode.v47.gffcompare.TestisLiverBrain.annotation.gtf


mkdir: cannot create directory ‘/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/newReference_Resconstructed’: File exists


## Get simplified annotation


In [13]:
## Author: Marta Espinosa Camarena
## Date: 21st May 24

## This scripts attemps to build a simplified version of an annotation file.
## By simplified we mean that we keep only 1 transcript per gene, being the longest transcript the selected one.
## Running mode:
## python get_simplified_annotation.py $absolute_path_to_annotation $outdir
## make sure the output directory has writing permissions

import pandas as pd
import sys, os
from pathlib import Path

##----------------- importing requested data -----------------##
# annotation_GTF = sys.argv[1]
annotation_GTF = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.gtf" ## testing

# outdir = sys.argv[2]
outdir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/newReference_Resconstructed"
outfile = os.path.basename(annotation_GTF)
outfile = outfile[:-4]

## reading reference file and separate by transcripts and genes
reference = pd.read_csv(annotation_GTF, sep="\t", header=None, comment="#")
reference['transcript_id'] = reference[8].str.extract('transcript_id "([^"]*)"', expand=True)
reference['gene_id'] = reference[8].str.extract('gene_id "([^"]*)"', expand=True)

In [14]:
reference['transcript_type'] = reference[8].str.extract('transcript_type "([^"]*)"', expand=True)
reference['gene_type'] = reference[8].str.extract('gene_type "([^"]*)"', expand=True)

## subset exons and calculate length
exons = reference[reference[2] == "exon"]
exons['length'] = exons[4] - exons[3]
genes = reference[reference[2] == "gene"]

## calculate length transcripts
length_tid = exons.groupby(['transcript_id'], as_index=False)[['length']].sum()


## make dataframe with geneid_transcriptid_length
ref_w_length = length_tid.merge(reference, on=["transcript_id"], how="right").drop_duplicates()
# ref_w_length = ref_w_length[['transcript_id','gene_id','length']]
# ref_w_length.drop_duplicates(inplace=True)
# ref_w_length.dropna(inplace=True)

## select longest transcript per gene
## Group by 'gene_id' and get the indices of the rows with the maximum 'length'
## select datafrem rows corresponding to longest trancript_id - if different transcripts for the same gene with same length, we keep the first one. 
ref_w_length
## For protein_coding genes we keep the longest protein coding transcript
filtered_ref_w_length = ref_w_length[
    (ref_w_length['gene_type'] != 'protein_coding') | 
    ((ref_w_length['gene_type'] == 'protein_coding') & (ref_w_length['transcript_type'] == 'protein_coding'))
]

max_length_indices = filtered_ref_w_length.groupby('gene_id')['length'].idxmax()

/tmp/ipykernel_1959167/101842013.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exons['length'] = exons[4] - exons[3]


In [15]:
longest_transcripts_gid = ref_w_length.loc[max_length_indices]
longest_transcripts_gid

,transcript_id,length,0,1,2,3,4,5,6,7,8,gene_id,transcript_type,gene_type
84487,ENST00000612152.4,3789.0,chrX,ENSEMBL,transcript,100627109,100637104,.,-,.,"gene_id ""ENSG00000000003.16""; transcript_id ""E...",ENSG00000000003.16,protein_coding,protein_coding
84459,ENST00000373031.5,1198.0,chrX,HAVANA,transcript,100584936,100599885,.,+,.,"gene_id ""ENSG00000000005.6""; transcript_id ""EN...",ENSG00000000005.6,protein_coding,protein_coding
3946569,ENST00000371582.8,1151.0,chr20,HAVANA,exon,50934867,50935236,.,-,.,"gene_id ""ENSG00000000419.14""; transcript_id ""E...",ENSG00000000419.14,protein_coding,protein_coding
436469,ENST00000367771.11,6295.0,chr1,HAVANA,exon,169849631,169853772,.,-,.,"gene_id ""ENSG00000000457.14""; transcript_id ""E...",ENSG00000000457.14,protein_coding,protein_coding
436221,ENST00000286031.10,4331.0,chr1,HAVANA,exon,169795409,169795983,.,+,.,"gene_id ""ENSG00000000460.17""; transcript_id ""E...",ENSG00000000460.17,protein_coding,protein_coding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95227,TCONS_00001174,491.0,chrX,StringTie,transcript,109054114,109054605,.,-,.,"transcript_id ""TCONS_00001174""; gene_id ""XLOC_...",XLOC_001035,NaN,NaN
100636,TCONS_00001181,216.0,chrX,StringTie,exon,119077902,119078118,.,-,.,"transcript_id ""TCONS_00001181""; gene_id ""XLOC_...",XLOC_001036,NaN,NaN
135923,TCONS_00001173,314.0,chrX,StringTie,transcript,155419406,155419720,.,-,.,"transcript_id ""TCONS_00001173""; gene_id ""XLOC_...",XLOC_001037,NaN,NaN
141020,TCONS_00001183,321.0,chrY,StringTie,transcript,9262142,9262463,.,+,.,"transcript_id ""TCONS_00001183""; gene_id ""XLOC_...",XLOC_001038,NaN,NaN


In [16]:
longest_transcripts_gid = longest_transcripts_gid[['gene_id','transcript_id','length']]
longest_transcripts_gid.drop_duplicates(inplace=True) ## no duplications

reference_longest = longest_transcripts_gid.merge(ref_w_length[ref_w_length[2] != "gene"], on=['gene_id','transcript_id','length'], how="left")


full_reference = pd.concat([reference_longest, genes])
full_reference.drop(['transcript_id','gene_id','length','transcript_type','gene_type'], axis=1, inplace=True)
full_reference.dropna(inplace=True)
full_reference.to_csv(outdir+"/"+outfile+".1transcript.gtf", sep="\t", header=None, index=None,  quoting = 3)

/tmp/ipykernel_1959167/1651782170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  longest_transcripts_gid.drop_duplicates(inplace=True) ## no duplications


In [17]:
full_reference

,0,1,2,3,4,5,6,7,8
0,chrX,ENSEMBL,transcript,100627109,100637104,.,-,.,"gene_id ""ENSG00000000003.16""; transcript_id ""E..."
1,chrX,ENSEMBL,exon,100627109,100629986,.,-,.,"gene_id ""ENSG00000000003.16""; transcript_id ""E..."
2,chrX,ENSEMBL,three_prime_utr,100627109,100629986,.,-,.,"gene_id ""ENSG00000000003.16""; transcript_id ""E..."
3,chrX,ENSEMBL,exon,100630759,100630866,.,-,.,"gene_id ""ENSG00000000003.16""; transcript_id ""E..."
4,chrX,ENSEMBL,three_prime_utr,100630759,100630797,.,-,.,"gene_id ""ENSG00000000003.16""; transcript_id ""E..."
...,...,...,...,...,...,...,...,...,...
4114863,KI270751.1,HAVANA,gene,2579,73732,.,-,.,"gene_id ""ENSG00000303867.1""; ID ""ENSG000003038..."
4114911,KI270751.1,HAVANA,gene,34096,71851,.,+,.,"gene_id ""ENSG00000303902.1""; ID ""ENSG000003039..."
4115234,KI270751.1,HAVANA,gene,133801,139253,.,-,.,"gene_id ""ENSG00000306528.1""; ID ""ENSG000003065..."
4115241,KI270753.1,HAVANA,gene,43135,44491,.,+,.,"gene_id ""ENSG00000297844.1""; ID ""ENSG000002978..."


In [18]:
%%bash -s "$users_dir" "$specie"

export PATH=/data/genomics/marta/tools/gff3sort-master:$PATH


reference=$1/$2/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.gtf
out=$1/$2/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.gtf


gff3sort.pl --precise --chr_order natural $reference > $out


## Quantification


In [19]:
%%bash -s "$users_dir" "$specie"

# sed 's/^chr//g' $1/$2/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.gtf > $1/$2/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.NOchr.gtf
# AnnotGTF=$1/$2/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.NOchr.gtf
## 1 transcript
sed 's/^chr//g' $1/$2/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.gtf > $1/$2/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.gtf
AnnotGTF=$1/$2/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.gtf

## formatting gtf - Cova's fix
sed 's/""/"/g' $AnnotGTF > $1/$2/newReference_Resconstructed/test.gtf
sed 's/\(transcript_id "[^"]*\).*/\1"/' $1/$2/newReference_Resconstructed/test.gtf > $1/$2/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.fixed.gtf
rm $1/$2/newReference_Resconstructed/test.gtf

In [20]:
%%bash -s "$users_dir" "$specie" "$prev_users_dir"

###PREPARING NEEDED DATA
# PROJECT=$1
# DIR=$2/$PROJECT
# p=$3 #single-end or paired-end dataset
p="single-end"
strand="secondstrand"
CLUSTERDIR=$1/$2/featureCounts_gffcompare
mkdir $CLUSTERDIR

# AnnotGTF=$1/$2/newReference_Resconstructed/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.gtf
AnnotGTF=$1/$2/newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.fixed.gtf

BAMDIR=$1/$2/STAR

module load Subread/2.0.3
########################

# countReadPairs may need to be removed in case of single-end reads

if [ $p == "paired-end" ]; then
    if [ $strand == "firststrand" ]; then
        featureCounts -T 10 -p -s 2 -g transcript_id -O --countReadPairs -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    elif [ $strand == "secondstrand" ]; then
        featureCounts -T 10 -p -s 1 -g transcript_id -O --countReadPairs -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    elif [ $strand == "unstranded" ]; then
        featureCounts -T 10 -p -s 0 -g transcript_id -O --countReadPairs -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    fi
fi
if [ $p == "single-end" ]; then
    if [ $strand == "firststrand" ]; then
        featureCounts -T 10 -s 2 -g transcript_id -O -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    elif [ $strand == "secondstrand" ]; then
        featureCounts -T 10 -s 1 -g transcript_id -O -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    elif [ $strand == "unstranded" ]; then
        featureCounts -T 10 -s 0 -g transcript_id -O -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    fi
fi



mkdir: cannot create directory ‘/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/v47/human/featureCounts_gffcompare’: File exists

        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.3

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 9 BAM files                                      ||
||                                                                            ||
||                           human_brain_rna_1Aligned.sortedByCoo

## TPMs

In [21]:
from rna_seq_normalization import Normalization as Norm

In [22]:
table_of_counts = pd.DataFrame()
### modify headers

df=pd.read_csv(os.path.join(users_dir,specie,"featureCounts_gffcompare/gffcompare_stranded_featureCounts.txt"), skiprows=1, sep="\t")
df['Geneid']=df['Geneid'].str.split('.').str[0]
filter_col = [col for col in df if col.startswith('/')]
for col in filter_col:
    new_col=col.split("Aligned")[0]
    new_col=new_col.split("/")[-1]

    df.rename(columns={col:new_col}, inplace=True)
    
cols_to_select = ["Geneid","Chr","Start","End","Strand","Length"]

print(len(df.columns))
df.rename(columns={'Geneid':'gene_id'}, inplace=True)

if table_of_counts.empty:
    table_of_counts = df
else:
    table_of_counts = table_of_counts.merge(df, on=["gene_id","Chr","Start","End","Strand","Length"])

table_of_counts.to_csv(os.path.join(users_dir,specie,"featureCounts_gffcompare/table_of_counts.csv"), index=None)

# get TPMs
length = table_of_counts['Length']
genes = table_of_counts['gene_id']
# we are only interested in the columns with counts
counts = table_of_counts
counts.drop(["Chr","Start","End","Strand","Length","gene_id"],axis=1, inplace=True)
# calculate TPMs
tpm_df = Norm.tpm(counts, length)
# add gene_names again
tpms = pd.concat([genes,tpm_df], axis=1)
tpms.to_csv(os.path.join(users_dir,specie,"featureCounts_gffcompare/table_of_counts_TPMs.csv"), index=None)
# add length again
tpms_length = pd.concat([genes,length,tpm_df], axis=1)
tpms_length.to_csv(os.path.join(users_dir,specie,"featureCounts_gffcompare/table_of_counts_TPMs_withLength.csv"), index=None)

15


## Run RiboNovel with the same annotation

## Generate annotation file

In [23]:
gtf=pd.read_csv(os.path.join(users_dir,specie,"newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.gtf"), sep="\t", header=None)
## keep only transcripts
gtf = gtf[gtf[2] == "transcript"]
gtf

/tmp/ipykernel_1959167/4029716907.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf=pd.read_csv(os.path.join(users_dir,specie,"newReference_Resconstructed/gencode.v47.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.gtf"), sep="\t", header=None)


,0,1,2,3,4,5,6,7,8
1,M,StringTie,transcript,1,577,.,-,.,"transcript_id ""TCONS_00001163""; gene_id ""XLOC_..."
3,M,ENSEMBL,transcript,577,647,.,+,.,"gene_id ""ENSG00000210049.1""; transcript_id ""EN..."
7,M,ENSEMBL,transcript,648,1601,.,+,.,"gene_id ""ENSG00000211459.2""; transcript_id ""EN..."
8,M,StringTie,transcript,683,1035,.,-,.,"transcript_id ""TCONS_00001164""; gene_id ""XLOC_..."
11,M,ENSEMBL,transcript,1602,1670,.,+,.,"gene_id ""ENSG00000210077.1""; transcript_id ""EN..."
...,...,...,...,...,...,...,...,...,...
782862,KI270751.1,HAVANA,transcript,34175,73722,.,-,.,"gene_id ""ENSG00000303867.1""; transcript_id ""EN..."
782865,KI270751.1,HAVANA,transcript,44958,53274,.,+,.,"gene_id ""ENSG00000303902.1""; transcript_id ""EN..."
782871,KI270751.1,HAVANA,transcript,133801,139253,.,-,.,"gene_id ""ENSG00000306528.1""; transcript_id ""EN..."
782875,KI270753.1,HAVANA,transcript,43135,44491,.,+,.,"gene_id ""ENSG00000297844.1""; transcript_id ""EN..."


In [24]:
## novels
gtf_novels = gtf[gtf[8].str.contains("TCONS")]
gtf_novels['transcript_id'] = gtf_novels[8].str.split(";", expand=True)[0]
gtf_novels['transcript_id'] = gtf_novels['transcript_id'].str.split('"', expand=True)[1]

gtf_novels['gene_id'] = gtf_novels[8].str.split(";", expand=True)[1]
gtf_novels['gene_id'] = gtf_novels['gene_id'].str.split('"', expand=True)[1]

gtf_novels['gene_name'] = gtf_novels['gene_id']
gtf_novels['transcript_type'] = "novel"
gtf_novels['gene_type'] = "novel"
gtf_novels['chr'] = gtf_novels[0]

annotation_novels = gtf_novels[['chr','transcript_id','gene_id','gene_name','transcript_type','gene_type']]
annotation_novels.drop_duplicates(inplace=True)

## remove novels
gtf_annotated = gtf[~gtf[8].str.contains("TCONS")]
gtf_annotated['gene_id'] = gtf_annotated[8].str.split(";", expand=True)[0]
gtf_annotated['gene_id'] = gtf_annotated['gene_id'].str.split('"', expand=True)[1]

gtf_annotated['transcript_id'] = gtf_annotated[8].str.split(";", expand=True)[1]
gtf_annotated['transcript_id'] = gtf_annotated['transcript_id'].str.split('"', expand=True)[1]

gtf_annotated['gene_name'] = gtf_annotated[8].str.split("gene_name", expand=True)[1]
gtf_annotated['gene_name'] = gtf_annotated['gene_name'].str.split('"', expand=True)[1]

gtf_annotated['transcript_type'] = gtf_annotated[8].str.split("transcript_type", expand=True)[1]
gtf_annotated['transcript_type'] = gtf_annotated['transcript_type'].str.split('"', expand=True)[1]

gtf_annotated['gene_type'] = gtf_annotated[8].str.split("gene_type", expand=True)[1]
gtf_annotated['gene_type'] = gtf_annotated['gene_type'].str.split('"', expand=True)[1]

gtf_annotated['chr'] = gtf_annotated[0]

annotation_annotated = gtf_annotated[['chr','transcript_id','gene_id','gene_name','transcript_type','gene_type']]
annotation_annotated.drop_duplicates(inplace=True)

annotation = pd.concat([annotation_annotated, annotation_novels])
annotation.to_csv(os.path.join(users_dir,specie,"newReference_Resconstructed/transID_geneID_isoforms_all.1to1.csv"), index=None)

###### lncRNA | processed pseudogene | novel | protein coding 
types=["lncRNA","processed_pseudogene","novel","protein_coding"]
annotation_selected = annotation[annotation['transcript_type'].isin(types)]
annotation_selected.to_csv(os.path.join(users_dir,specie,"newReference_Resconstructed/transID_geneID_isoforms_selected_version.1to1.csv"), index=None)

## no version
annotation_selected_v = annotation_selected
annotation_selected_v['gene_id'] = annotation_selected_v['gene_id'].str.split(".", expand=True)[0]
annotation_selected_v['transcript_id'] = annotation_selected_v['transcript_id'].str.split(".", expand=True)[0]
annotation_selected_v.to_csv(os.path.join(users_dir,specie,"newReference_Resconstructed/transID_geneID_isoforms_selected.1to1.csv"), index=None)


/tmp/ipykernel_1959167/2607630529.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gtf_novels['transcript_id'] = gtf_novels[8].str.split(";", expand=True)[0]
/tmp/ipykernel_1959167/2607630529.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gtf_novels['transcript_id'] = gtf_novels['transcript_id'].str.split('"', expand=True)[1]
/tmp/ipykernel_1959167/2607630529.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va